In [16]:
using SQLite
import SQLite.Stmt
import SQLite.DBInterface.execute
using Distributions

In [32]:
# Functions to create fake normal data in lieu of actual simulation runs.

function generate_runs(db, combo_id, n_runs)
    execute(db, "CREATE TABLE IF NOT EXISTS runs (run_id INTEGER PRIMARY KEY AUTOINCREMENT, combo_id)")
    stmt = Stmt(db, "INSERT INTO runs (combo_id) VALUES (?)")
    for i in 1:n_runs
        execute(stmt, [combo_id])
    end

end

function get_run_ids(db, combo_id)
    [run_id for (run_id,) in execute(db, "SELECT run_id FROM runs WHERE combo_id = ?", [combo_id])]
end

function generate_scalar(db, combo_id, tblname, mean, std)
    execute(db, "CREATE TABLE IF NOT EXISTS $(tblname) (run_id, value)")
    stmt = Stmt(db, "INSERT INTO $(tblname) VALUES (?,?)")

    run_ids = get_run_ids(db, combo_id)
    x = rand(Normal(mean, std), length(run_ids))
    for (i, run_id,) in enumerate(run_ids)
        execute(stmt, [run_id, x[i]])
    end
end

function generate_time_series(db, combo_id, tblname, ts, mean, std)
    execute(db, "CREATE TABLE IF NOT EXISTS $(tblname) (run_id, time, value)")
    stmt = Stmt(db, "INSERT INTO $(tblname) VALUES (?,?,?)")

    run_ids = get_run_ids(db, combo_id)
    x = rand(Normal(mean, std), (length(ts), length(run_ids)))
    for (j, run_id,) in enumerate(run_ids)
        for (i, t,) in enumerate(ts)
            execute(stmt, (run_id, t, x[i, j]))
        end
    end
end

generate_time_series (generic function with 1 method)

In [68]:
# Generate fake data for two different parameter combinations, 20 runs,
# one scalar value, one time series.
# Scenario: an attempt at improving performance.
# Performance improves, and there is no change in behavior for parameter combination 1.
# However, a bug is introduced that shows up for parameter combination 2.

db1 = SQLite.DB(":memory:")
db2 = SQLite.DB(":memory:")

ts = 0:30:720

# Parameter combination 1
generate_runs(db1, 1, 20)
generate_runs(db2, 1, 20)
generate_scalar(db1, 1, "elapsed_time", 100.0, 10.0) # elapsed_time for code version 1
generate_scalar(db2, 1, "elapsed_time",  80.0, 10.0)  # elapsed_time for code version 2 (faster)
generate_time_series(db1, 1, "moi_mean", ts, 3, 0.1)   # moi_mean for code version 1
generate_time_series(db2, 1, "moi_mean", ts, 3, 0.1)   # moi_mean for code version 2 (same)

# Parameter combination 2
generate_runs(db1, 2, 20)
generate_runs(db2, 2, 20)
generate_scalar(db1, 2, "elapsed_time", 120.0, 15.0) # elapsed_time for code version 1
generate_scalar(db2, 2, "elapsed_time", 95.0, 15.0)  # elapsed_time for code version 2 (faster)
generate_time_series(db1, 2, "moi_mean", ts, 2, 0.1)   # moi_mean for code version 1
generate_time_series(db2, 2, "moi_mean", ts, 1.8, 0.1)   # moi_mean for code version 2 (DIFFERENT - A BUG)

In [40]:
[(run_id, time, value) for (run_id, time, value) in execute(db1, "SELECT * FROM elapsed_time")]

40-element Vector{Tuple{Float64, Float64}}:
 (1.0, 96.10739132606024)
 (2.0, 96.43390891319504)
 (3.0, 90.68779626106664)
 (4.0, 97.0837713852988)
 (5.0, 88.00097196150526)
 (6.0, 100.28064880702108)
 (7.0, 105.41128918764174)
 (8.0, 107.21876432068245)
 (9.0, 106.54120590419353)
 (10.0, 94.70406190474748)
 ⋮
 (32.0, 103.27127021209616)
 (33.0, 147.47591302233732)
 (34.0, 102.81179306493615)
 (35.0, 133.4888767628679)
 (36.0, 127.76579121334862)
 (37.0, 107.29684556508111)
 (38.0, 130.10599445732956)
 (39.0, 112.33961617510523)
 (40.0, 125.17763155633828)

In [41]:
[(run_id, time, value) for (run_id, time, value) in execute(db1, "SELECT * FROM moi_mean")]

1000-element Vector{Tuple{Int64, Int64, Float64}}:
 (1, 0, 3.085437204780984)
 (1, 30, 3.01491826337338)
 (1, 60, 3.002151546959092)
 (1, 90, 2.9686276157402416)
 (1, 120, 2.8957891280249024)
 (1, 150, 2.925763690134143)
 (1, 180, 2.990693215371097)
 (1, 210, 2.827404412939092)
 (1, 240, 2.8597301025725472)
 (1, 270, 3.0422095866029077)
 ⋮
 (40, 480, 2.126196030443726)
 (40, 510, 1.9458964826059553)
 (40, 540, 2.0179596544199563)
 (40, 570, 1.9275809627277747)
 (40, 600, 1.9477441916540292)
 (40, 630, 2.080536207619095)
 (40, 660, 1.8679463698567846)
 (40, 690, 1.7712294594445743)
 (40, 720, 1.7590007106519683)

In [71]:
import StatsAPI.pvalue
import StatsBase.mean
import StatsBase.mad
import HypothesisTests.ApproximateTwoSampleKSTest

# Functions to compare using K-S tests

"""
    Do K-S tests using SQLite queries that take one parameter, run_id.

    `db1` and `db2` are assumed to have identical parameter combinations labeled the same way,
    via the `combo_id` column in the `runs` table.

    The query should return a single value for the provided run_id.

    If values from db2 need to be extracted with a different query, provide a value for `q2`.
"""
function compare(db1, db2, q1; q2 = q1)
    combo_ids = get_combo_ids(db1)
    @assert all(combo_ids .== get_combo_ids(db2))
    [(combo_id, compare(db1, db2, combo_id, q1; q2 = q2)) for combo_id in combo_ids]
end

function get_combo_ids(db)
    [combo_id for (combo_id,) in execute(db, "SELECT DISTINCT combo_id FROM runs ORDER BY combo_id")]
end

function compare(db1, db2, combo_id, q1; q2 = q1)
    v1 = query_values_for_combo_id(db1, combo_id, q1)
    mean1 = mean(v1)
    mad1 = mad(v1; center = mean1)

    v2 = query_values_for_combo_id(db2, combo_id, q2)
    mean2 = mean(v2)
    mad2 = mad(v2; center = mean2)

    (ApproximateTwoSampleKSTest(v1, v2), (mean1, mad1), (mean2, mad2))
end

function query_values_for_combo_id(db, combo_id, q)
    stmt = Stmt(db, q)
    run_ids = get_run_ids(db, combo_id)

    [query_value_for_run_id(stmt, run_id) for run_id in run_ids]
end

function query_value_for_run_id(stmt, run_id)
    for (value,) in execute(stmt, (run_id,))
        return value
    end
end

query_value_for_run_id (generic function with 1 method)

In [74]:
for (combo_id, (test, (mean1, mad1), (mean2, mad2))) in compare(db1, db2, "SELECT value FROM elapsed_time WHERE run_id = ?")
    println("Testing elapsed time for parameter combination $(combo_id)...")
    println("    (mean1 $(mean1), mad1 $(mad1))")
    println("    (mean2 $(mean2), mad2 $(mad2))")
    println("    p-value: $(pvalue(test))")
    if pvalue(test) < 0.01
        println("    DIFFERENT distributions w/ p < 0.01")
    else
        println("    undetectable difference between distributions w/ p < 0.01")
    end
end

Testing elapsed time for parameter combination 1...
    (mean1 98.52747830909739, mad1 12.75526719689743)
    (mean2 82.05364853311593, mad2 12.779490311421853)
    p-value: 0.0014931716161319915
    DIFFERENT distributions w/ p < 0.01
Testing elapsed time for parameter combination 2...
    (mean1 116.75995982764493, mad1 11.602721168536982)
    (mean2 93.7473968405003, mad2 23.298801787044546)
    p-value: 0.004715723951164094
    DIFFERENT distributions w/ p < 0.01


In [77]:
for (combo_id, (test, (mean1, mad1), (mean2, mad2))) in compare(
        db1, db2,
        "SELECT AVG(value) FROM moi_mean WHERE run_id = ? AND time >= 360 AND time <= 720"
    )
    println("Testing mean moi over time for parameter combination $(combo_id)...")
    println("    (mean1 $(mean1), mad1 $(mad1))")
    println("    (mean2 $(mean2), mad2 $(mad2))")
    println("    p-value: $(pvalue(test))")
    if pvalue(test) < 0.01
        println("    DIFFERENT distributions w/ p < 0.01")
    else
        println("    undetectable difference between distributions w/ p < 0.01")
    end
end

Testing mean moi over time for parameter combination 1...
    (mean1 3.013226253800627, mad1 0.03880718915594512)
    (mean2 2.996645512220396, mad2 0.020749624789856785)
    p-value: 0.17247627033056145
    undetectable difference between distributions w/ p < 0.01
Testing mean moi over time for parameter combination 2...
    (mean1 2.0020889929805468, mad1 0.02438158593309094)
    (mean2 1.815571544166526, mad2 0.03579324777309966)
    p-value: 4.122307244877057e-9
    DIFFERENT distributions w/ p < 0.01
